# Database Connection & Examples

This notebook demonstrates:
1. **Connect** to the red-data PostGIS database
2. **List** all available tables (from documentation)
3. **Load** example fact table
4. **Join** fact table with reference table
5. **Spatial join** using PostGIS functions


In [20]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
from urllib.parse import quote_plus


## 1. Database Connection


In [21]:
# Database credentials
DB_HOST = "dokploy.red-data.eu"
DB_PORT = 54321
DB_NAME = "red-data-db"
DB_USER = "zensus_user"
DB_PASSWORD = "kiskIv-kehcyh-hishu4"

connection_string = f"postgresql://{DB_USER}:{quote_plus(DB_PASSWORD)}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=disable"
engine = create_engine(connection_string)

print("✅ Connected to database")
print(f"   Host: {DB_HOST}:{DB_PORT}")
print(f"   Database: {DB_NAME}")


✅ Connected to database
   Host: dokploy.red-data.eu:54321
   Database: red-data-db


## 2. List All Available Tables

These are the meaningful tables documented in `DATABASE_TABLES_DOCUMENTATION.md`:


In [22]:
# Get all tables in zensus schema
tables_query = """
SELECT 
    tablename
FROM pg_tables
WHERE schemaname = 'zensus'
ORDER BY tablename;
"""

tables_df = pd.read_sql(tables_query, engine)

# Categorize tables in Python (safer than SQL CASE)
def categorize_table(name):
    if name.startswith('ref_'):
        return '📍 Reference'
    elif '_100m_' in name:
        return '📊 Fact (100m)'
    elif '_1km_' in name:
        return '📊 Fact (1km)'
    elif '_10km_' in name:
        return '📊 Fact (10km)'
    else:
        return '❓ Other'

tables_df['category'] = tables_df['tablename'].apply(categorize_table)

# Display grouped by category
print("="*80)
print("AVAILABLE TABLES IN ZENSUS SCHEMA")
print("="*80)

for category in ['📍 Reference', '📊 Fact (100m)', '📊 Fact (1km)', '📊 Fact (10km)', '❓ Other']:
    category_tables = tables_df[tables_df['category'] == category]
    if len(category_tables) > 0:
        print(f"\n{category} ({len(category_tables)} tables):")
        for _, row in category_tables.iterrows():
            print(f"  • {row['tablename']}")

print(f"\n{'='*80}")
print(f"Total: {len(tables_df)} tables")
print("="*80)


AVAILABLE TABLES IN ZENSUS SCHEMA

📍 Reference (8 tables):
  • ref_county
  • ref_electoral_district
  • ref_federal_state
  • ref_grid_100m
  • ref_grid_10km
  • ref_grid_1km
  • ref_lwu_properties
  • ref_municipality

📊 Fact (100m) (41 tables):
  • fact_zensus_100m_alter_5altersklassen
  • fact_zensus_100m_alter_in_10er_jahresgruppen
  • fact_zensus_100m_alter_infr
  • fact_zensus_100m_anteil_auslaender
  • fact_zensus_100m_anteil_ueber_65
  • fact_zensus_100m_anteil_unter_18
  • fact_zensus_100m_auslaenderanteil_ab18
  • fact_zensus_100m_baujahr_jz
  • fact_zensus_100m_bevoelkerungszahl
  • fact_zensus_100m_deutsche_staatsangehoerige_ab18
  • fact_zensus_100m_durchschn_flaeche_je_bewohner
  • fact_zensus_100m_durchschn_flaeche_je_wohnung
  • fact_zensus_100m_durchschn_haushaltsgroesse
  • fact_zensus_100m_durchschn_nettokaltmiete
  • fact_zensus_100m_durchschnittliche_nettokaltmiete_und_anzahl_de
  • fact_zensus_100m_durchschnittsalter
  • fact_zensus_100m_eigentuemerquote
  • fact

## 3. Load Example Fact Table

Let's load the **rent data** (one of our most-used fact tables):


In [27]:
# Load a sample of rent data (limit to 1000 rows for quick display)
rent_query = """
SELECT 
    grid_id,
    year,
    durchschnmieteqm as avg_rent_per_sqm,
    anzahlwohnungen as num_flats,
    x_mp_100m,
    y_mp_100m
FROM zensus.fact_zensus_100m_durchschnittliche_nettokaltmiete_und_anzahl_der_wohnungen
WHERE durchschnmieteqm IS NOT NULL 
    AND anzahlwohnungen > 0
ORDER BY anzahlwohnungen DESC
LIMIT 1000;
"""

rent_df = pd.read_sql(rent_query, engine)

print(f"✅ Loaded {len(rent_df)} rent records")
print(f"\nColumns: {list(rent_df.columns)}")
print(f"\nSample statistics:")
print(f"  Average rent: €{rent_df['avg_rent_per_sqm'].mean():.2f}/m²")
print(f"  Average flats per grid: {rent_df['num_flats'].mean():.1f}")
print(f"  Max flats in a grid: {rent_df['num_flats'].max()}")

rent_df.head()


✅ Loaded 1000 rent records

Columns: ['grid_id', 'year', 'avg_rent_per_sqm', 'num_flats', 'x_mp_100m', 'y_mp_100m']

Sample statistics:
  Average rent: €9.60/m²
  Average flats per grid: 259.3
  Max flats in a grid: 734


,grid_id,year,avg_rent_per_sqm,num_flats,x_mp_100m,y_mp_100m
0,CRS3035RES100mN3270650E4559150,2022,8.83,734,4559150.0,3270650.0
1,CRS3035RES100mN2921850E4224350,2022,21.30,681,4224350.0,2921850.0
2,CRS3035RES100mN3094550E4105450,2022,11.97,672,4105450.0,3094550.0
3,CRS3035RES100mN2783350E4441450,2022,15.88,536,4441450.0,2783350.0
4,CRS3035RES100mN3275950E4536150,2022,6.07,519,4536150.0,3275950.0


## 3.1 Load LWU Statistics Table


In [28]:
# Load data with geometry for mapping
lwu_stats_gdf = gpd.read_postgis(
    "SELECT * FROM analytics.view_lwu_stats_geo",
    engine,
    geom_col='geom'
)

lwu_stats_gdf

,property_id,weighted_avg_rent_per_sqm,rent_total_flats,heating_total_buildings,heating_fernheizung_pct,heating_etagenheizung_pct,heating_blockheizung_pct,heating_zentralheizung_pct,heating_einzel_mehrraumoefen_pct,heating_keine_heizung_pct,...,baujahr_vor1919_pct,baujahr_a1919bis1948_pct,baujahr_a1949bis1978_pct,baujahr_a1979bis1990_pct,baujahr_a1991bis2000_pct,baujahr_a2001bis2010_pct,baujahr_a2011bis2019_pct,baujahr_a2020undspaeter_pct,created_at,geom
0,lwu_fls.11058003400457,9.939922,37.957321,39.530009,0.962549,0.030452,0.00,0.007000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.051243,0.936978,0.011779,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4562122.356 3279542.406, 45621..."
1,lwu_fls.11004505500196,6.461483,0.158280,0.142037,0.000000,0.457452,0.00,0.542548,0.0,0.0,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4549303.207 3268257.33, 454934..."
2,lwu_fls.11006230500228,6.974226,12.935165,15.148929,0.790810,0.000000,0.00,0.209190,0.0,0.0,...,0.000000,0.0,0.713919,0.255154,0.000000,0.030927,0.000000,0.000000,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4556065.147 3263831.759, 45560..."
3,lwu_fls.11004505500186,6.510000,1.190445,1.050393,0.000000,0.533333,0.00,0.466667,0.0,0.0,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4549234.167 3268167.993, 45492..."
4,lwu_fls.11058100100744,9.419827,64.481664,65.684623,0.453284,0.000000,0.00,0.546716,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.099774,0.900226,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4562291.034 3279732.212, 45622..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5463,lwu_fls.11051544400362,5.800000,3.918129,4.010320,1.000000,0.000000,0.00,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4564374.831 3265341.244, 45643..."
5464,lwu_fls.11051544400525,7.010000,5.753460,5.585884,0.000000,0.000000,0.48,0.520000,0.0,0.0,...,0.428571,0.0,0.000000,0.000000,0.571429,0.000000,0.000000,0.000000,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4564553.343 3265035.318, 45645..."
5465,lwu_fls.11051545604347,6.669667,13.686811,14.345184,0.476468,0.021671,0.00,0.501861,0.0,0.0,...,0.308042,0.0,0.000000,0.464333,0.000000,0.000000,0.227625,0.000000,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4563259.127 3265426.153, 45632..."
5466,lwu_fls.11051544400024,5.800000,1.654169,1.693090,1.000000,0.000000,0.00,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2026-01-07 10:35:40.105993,"MULTIPOLYGON (((4564357.722 3265387.184, 45643..."


## 4. Join Fact Table with Reference Table

Join **rent data** with **100m grid geometry** to get spatial data:


In [17]:
# Join rent data with grid geometry (SQL-based join, then load to GeoPandas)
join_query = """
SELECT 
    r.grid_id,
    r.durchschnmieteqm as avg_rent_per_sqm,
    r.anzahlwohnungen as num_flats,
    g.geom
FROM zensus.fact_zensus_100m_durchschnittliche_nettokaltmiete_und_anzahl_der_wohnungen r
INNER JOIN zensus.ref_grid_100m g ON r.grid_id = g.grid_id
WHERE r.durchschnmieteqm IS NOT NULL 
    AND r.anzahlwohnungen > 0
    AND r.durchschnmieteqm > 10  -- High rent areas only
ORDER BY r.durchschnmieteqm DESC
LIMIT 100;
"""

rent_with_geom_gdf = gpd.read_postgis(join_query, engine, geom_col='geom')

print(f"✅ Loaded {len(rent_with_geom_gdf)} grid cells with rent data + geometry")
print(f"\nDataFrame type: {type(rent_with_geom_gdf)}")
print(f"CRS: {rent_with_geom_gdf.crs}")
print(f"\nTop 5 most expensive grid cells:")
print(rent_with_geom_gdf[['grid_id', 'avg_rent_per_sqm', 'num_flats']].head())

# Display
rent_with_geom_gdf.head()


✅ Loaded 100 grid cells with rent data + geometry

DataFrame type: <class 'geopandas.geodataframe.GeoDataFrame'>
CRS: EPSG:3035

Top 5 most expensive grid cells:
                          grid_id  avg_rent_per_sqm  num_flats
0  CRS3035RES100mN3409150E4314950             49.69         13
1  CRS3035RES100mN3065350E4091650             49.39          6
2  CRS3035RES100mN2899550E4197050             48.81          3
3  CRS3035RES100mN3004850E4441350             48.70          3
4  CRS3035RES100mN3005350E4208750             48.63          8


,grid_id,avg_rent_per_sqm,num_flats,geom
0,CRS3035RES100mN3409150E4314950,49.69,13,"POLYGON ((4314900 3409100, 4314900 3409200, 43..."
1,CRS3035RES100mN3065350E4091650,49.39,6,"POLYGON ((4091600 3065300, 4091600 3065400, 40..."
2,CRS3035RES100mN2899550E4197050,48.81,3,"POLYGON ((4197000 2899500, 4197000 2899600, 41..."
3,CRS3035RES100mN3004850E4441350,48.70,3,"POLYGON ((4441300 3004800, 4441300 3004900, 44..."
4,CRS3035RES100mN3005350E4208750,48.63,8,"POLYGON ((4208700 3005300, 4208700 3005400, 42..."


## 5. Spatial Join Using PostGIS Functions

**Example:** Find which 100m grid cells intersect with LWU properties and calculate overlap ratio.

This demonstrates PostGIS spatial functions:
- `ST_Intersects()` - Test if geometries intersect
- `ST_Intersection()` - Get intersection geometry
- `ST_Area()` - Calculate area


In [18]:
# Spatial join: LWU properties with intersecting grid cells
spatial_join_query = """
SELECT 
    l.property_id,
    l.original_id,
    g.grid_id,
    -- Calculate overlap ratio: how much of the grid cell is covered by the property
    ST_Area(ST_Intersection(l.geom, g.geom)) / NULLIF(ST_Area(g.geom), 0) as overlap_ratio,
    -- Calculate property coverage: how much of the property is in this grid cell
    ST_Area(ST_Intersection(l.geom, g.geom)) / NULLIF(ST_Area(l.geom), 0) as property_coverage,
    g.geom
FROM zensus.ref_lwu_properties l
INNER JOIN zensus.ref_grid_100m g ON ST_Intersects(l.geom, g.geom)
WHERE ST_Area(ST_Intersection(l.geom, g.geom)) > 0
LIMIT 500;
"""

spatial_join_gdf = gpd.read_postgis(spatial_join_query, engine, geom_col='geom')

print(f"✅ Found {len(spatial_join_gdf)} property-grid intersections")
print(f"\nSample statistics:")
print(f"  Average overlap ratio: {spatial_join_gdf['overlap_ratio'].mean():.2%}")
print(f"  Average property coverage: {spatial_join_gdf['property_coverage'].mean():.2%}")
print(f"\nExample: One property might span multiple grid cells")

# Show one property's grid intersections
sample_property = spatial_join_gdf['property_id'].iloc[0]
sample_grids = spatial_join_gdf[spatial_join_gdf['property_id'] == sample_property]
print(f"\nProperty '{sample_property}' intersects with {len(sample_grids)} grid cells:")
print(sample_grids[['grid_id', 'overlap_ratio', 'property_coverage']])

spatial_join_gdf.head(10)


✅ Found 500 property-grid intersections

Sample statistics:
  Average overlap ratio: 19.05%
  Average property coverage: 26.08%

Example: One property might span multiple grid cells

Property 'lwu_fls.11005800101038' intersects with 9 grid cells:
                            grid_id  overlap_ratio  property_coverage
0    CRS3035RES100mN3259350E4552250       0.912566           0.315314
1    CRS3035RES100mN3259350E4552350       0.405601           0.140145
2    CRS3035RES100mN3259350E4552450       0.035024           0.012102
5    CRS3035RES100mN3259450E4552150       0.005942           0.002053
6    CRS3035RES100mN3259450E4552250       0.177071           0.061182
391  CRS3035RES100mN3259150E4552250       0.132636           0.045829
401  CRS3035RES100mN3259250E4552250       0.705060           0.243615
404  CRS3035RES100mN3259250E4552350       0.393405           0.135931
406  CRS3035RES100mN3259250E4552450       0.126845           0.043828


,property_id,original_id,grid_id,overlap_ratio,property_coverage,geom
0,lwu_fls.11005800101038,lwu_fls.11005800101038______,CRS3035RES100mN3259350E4552250,0.912566,0.315314,"POLYGON ((4552200 3259300, 4552200 3259400, 45..."
1,lwu_fls.11005800101038,lwu_fls.11005800101038______,CRS3035RES100mN3259350E4552350,0.405601,0.140145,"POLYGON ((4552300 3259300, 4552300 3259400, 45..."
2,lwu_fls.11005800101038,lwu_fls.11005800101038______,CRS3035RES100mN3259350E4552450,0.035024,0.012102,"POLYGON ((4552400 3259300, 4552400 3259400, 45..."
3,lwu_fls.11005800400133,lwu_fls.11005800400133______,CRS3035RES100mN3259450E4551150,0.093948,0.228783,"POLYGON ((4551100 3259400, 4551100 3259500, 45..."
4,lwu_fls.11005800400133,lwu_fls.11005800400133______,CRS3035RES100mN3259450E4551250,0.061797,0.150489,"POLYGON ((4551200 3259400, 4551200 3259500, 45..."
5,lwu_fls.11005800101038,lwu_fls.11005800101038______,CRS3035RES100mN3259450E4552150,0.005942,0.002053,"POLYGON ((4552100 3259400, 4552100 3259500, 45..."
6,lwu_fls.11005800101038,lwu_fls.11005800101038______,CRS3035RES100mN3259450E4552250,0.177071,0.061182,"POLYGON ((4552200 3259400, 4552200 3259500, 45..."
7,lwu_fls.11005800100638,lwu_fls.11005800100638______,CRS3035RES100mN3259450E4552850,0.298797,0.142174,"POLYGON ((4552800 3259400, 4552800 3259500, 45..."
8,lwu_fls.11005800400133,lwu_fls.11005800400133______,CRS3035RES100mN3259550E4551150,0.142000,0.345798,"POLYGON ((4551100 3259500, 4551100 3259600, 45..."
9,lwu_fls.11005800400133,lwu_fls.11005800400133______,CRS3035RES100mN3259550E4551250,0.112898,0.274930,"POLYGON ((4551200 3259500, 4551200 3259600, 45..."
